
# EDA — Consumo de Café y KPIs de Negocio

Este cuaderno realiza:
1. **Limpieza y normalización** de los datos (columnas de años tipo `1990/91` → `year` entero).
2. **Visualizaciones** para negocio (eje X con *todos los años* en 45°, etiquetas de datos).
3. **KPIs**: Ingresos, Costos, Utilidad, Márgenes, CAGR, Participación de mercado.
4. **Tablas** de resumen por país y tipo de café.
5. **Guardado** del dataset limpio para modelado: `data/coffee_clean.csv`.


In [ ]:

# %%
# Configuración e imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from utils.io import load_coffee_data, load_price_data, merge_coffee_price, save_clean_dataset
from utils.metrics import profit, margin, cagr, market_share

plt.rcParams["figure.figsize"] = (11, 5)
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

DATA_CONSUMO = "coffee_db.csv"  # ajusta las rutas si lo ejecutas fuera de este entorno
DATA_PRECIOS = "precios.csv"
OUT_CLEAN = "data/coffee_clean.csv"

Path("data").mkdir(exist_ok=True)


In [ ]:

# %%
# Carga y limpieza
coffee_long = load_coffee_data(DATA_CONSUMO)
price_yearly = load_price_data(DATA_PRECIOS)
df = merge_coffee_price(coffee_long, price_yearly)

# Supuestos de costos (ajustables)
FIXED_COST = 1_000_000.0   # costos fijos anuales globales (USD)
VAR_COST_PER_CUP = 0.05    # costo variable por taza (USD)

df["revenue"] = df["price"] * df["consumption"]
df["profit"] = profit(df["consumption"], df["price"], FIXED_COST/df["year"].nunique(), VAR_COST_PER_CUP)
df["margin"] = margin(df["consumption"], df["price"], FIXED_COST/df["year"].nunique(), VAR_COST_PER_CUP)

# Participación de mercado por año
df = market_share(df, value_col="consumption", group_cols=("year",))

# Guardar dataset limpio
clean_path = save_clean_dataset(df, OUT_CLEAN)
clean_path


In [ ]:

# %%
# Tablas de resumen
top_countries = (df.groupby("country", as_index=False)["consumption"]
                   .sum().sort_values("consumption", ascending=False)
                   .head(10))

summary_by_country_type = (df.groupby(["country","type"], as_index=False)
                             .agg(consumption_total=("consumption","sum"),
                                  revenue_total=("revenue","sum"),
                                  profit_total=("profit","sum"),
                                  margin_avg=("margin","mean")))

df.head(3), top_countries.head(10), summary_by_country_type.head(10)


In [ ]:

# %%
# Helpers de graficación con etiquetas y ticks anuales rotados 45°
def plot_line_with_labels(x, y, title, xlabel, ylabel):
    fig, ax = plt.subplots()
    ax.plot(x, y, marker="o")
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    # ticks de todos los años
    ax.set_xticks(sorted(df["year"].unique()))
    ax.tick_params(axis="x", rotation=45)
    # etiquetas de datos
    for xi, yi in zip(x, y):
        ax.annotate(f"{yi:,.0f}", (xi, yi), textcoords="offset points", xytext=(0,6), ha="center")
    plt.tight_layout()
    plt.show()

def plot_bar_with_labels(x, y, title, xlabel, ylabel):
    fig, ax = plt.subplots()
    ax.bar(x, y)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.tick_params(axis="x", rotation=45)
    # etiquetas
    for xi, yi in zip(x, y):
        ax.annotate(f"{yi:,.0f}", (xi, yi), textcoords="offset points", xytext=(0,6), ha="center")
    plt.tight_layout()
    plt.show()


In [ ]:

# %%
# Tendencia global del consumo
global_year = df.groupby("year", as_index=False)["consumption"].sum()
plot_line_with_labels(global_year["year"], global_year["consumption"],
                      "Consumo global por año", "Año", "Tazas")

# Top 10 países por consumo acumulado
plot_bar_with_labels(top_countries["country"], top_countries["consumption"],
                     "Top 10 países por consumo acumulado", "País", "Tazas")

# Ingresos globales por año
global_rev = df.groupby("year", as_index=False)["revenue"].sum()
plot_line_with_labels(global_rev["year"], global_rev["revenue"],
                      "Ingresos globales por año (USD)", "Año", "USD")


In [ ]:

# %%
# KPIs extra: CAGR por país (consumo) entre primer y último año observado
years = (df["year"].min(), df["year"].max())
periods = years[1] - years[0]
country_cagr = []
for c, g in df.groupby("country"):
    g = g.sort_values("year")
    first = g.loc[g["year"].idxmin(), "consumption"]
    last = g.loc[g["year"].idxmax(), "consumption"]
    country_cagr.append({"country": c, "cagr_consumption": cagr(first, last, periods)})
country_cagr = pd.DataFrame(country_cagr).sort_values("cagr_consumption", ascending=False)
country_cagr.head(10)
